In [9]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import h5py  
#import readers as rd
import os
import fiona
import geopandas
import time
from sklearn.neighbors import KernelDensity
import scipy
from astropy.time import Time
import geopandas as gpd

In [10]:
def Rescale(path, outdir, window_width):
    j=0
    for file in os.listdir(path):
        if file.endswith('shp'):
            filename = file[3:8]
            print(filename)
            points = os.path.join(path,file)
            pts = gpd.read_file(points)
            
                # Along Track Distance Values
            AT_dist_values = pts['AT_dist'].values
            AT_dist_totmax = AT_dist_values.max()
            AT_dist_totmin = AT_dist_values.min()
            
            lons= pts.geometry.x
            lats = pts.geometry.y
            
            SD_values = pts['SD'].values

                #Moving Window
            windows = np.arange(AT_dist_totmin, AT_dist_totmax, 100).tolist()

            print('windows',len(windows))
            
            a = np.empty(len(windows))
            b = np.empty(len(windows))  
            c = np.empty(len(windows))  
            d = np.empty(len(windows))  

            i = 0
            
            for window_center in windows:
                
                #Get minimum window boundries
                min_dist = window_center - window_width
                min_dist_array = np.where(AT_dist_values > min_dist)
                min_dist_row = min_dist_array[0][0]

                #Get maximum window boundries
                max_dist = window_center + window_width
                if max_dist < AT_dist_values[-1]:
                    max_dist_array = np.where(AT_dist_values > max_dist)
                    max_dist_row = max_dist_array[0][0]
                else:
                    max_dist = AT_dist_values[-1] 

                #Get window center lat & long for plotting later        
                idx = (np.abs(AT_dist_values - window_center)).argmin()
                lat = lats[idx]
                lon = lons[idx]
                    #print('df',dfATL03.shape)
                    
                window_SD = SD_values[min_dist_row:max_dist_row]
                #print('len',len(window_SD))
                if len(window_SD) > (window_width/20): #Normally 5
                    SD = np.median(window_SD)
                    
                    
                    a[i] = window_center #AT_DIST of window Center
                    b[i] = SD
                    c[i] = lat
                    d[i] = lon

                    i+=1

                else:
                    a[i] = window_center
                    b[i] = -999
                    c[i] = lat
                    d[i] = lon
                    i+=1
            
            df = gpd.GeoDataFrame(a, columns = ['AT_dist']) 
            df["SD"] = b
            df["lats"] = c
            df["lons"] = d
            
            gdf = geopandas.GeoDataFrame(
            df, geometry=geopandas.points_from_xy(df.lons, df.lats))
            gdf = gdf.set_crs('EPSG:25833')
            gdf = gdf[gdf['SD'] > 0]
            gdf = gdf[gdf['SD'] < 10]
            gdf = gdf[gdf['SD'] != -999]
            gdf.to_file(outdir + filename + '.shp')
            print(gdf)
    return df

In [11]:
path = 'C:/Users/Rasmu/Documents/Thesis/Snow_Depth/'
outdir = 'C:/Users/Rasmu/Documents/Thesis/SD_Rescaled/'
window_width = 500

In [12]:
df = Rescale(path, outdir, window_width)

gt1l1
windows 996
     AT_dist     SD          lats          lons                       geometry
284  28510.0  2.535  6.719409e+06  49634.140880  POINT (49634.141 6719409.224)
285  28610.0  2.395  6.719399e+06  49631.867464  POINT (49631.867 6719399.493)
286  28710.0  2.400  6.719301e+06  49608.975370  POINT (49608.975 6719301.483)
356  35710.0  2.760  6.712350e+06  48004.742480  POINT (48004.742 6712349.847)
357  35810.0  2.930  6.712350e+06  48004.742480  POINT (48004.742 6712349.847)
..       ...    ...           ...           ...                            ...
939  94010.0  5.610  6.655568e+06  34757.700788  POINT (34757.701 6655568.240)
940  94110.0  5.360  6.655500e+06  34741.711849  POINT (34741.712 6655499.802)
941  94210.0  5.360  6.655500e+06  34741.711849  POINT (34741.712 6655499.802)
942  94310.0  5.590  6.655188e+06  34668.563588  POINT (34668.564 6655187.820)
943  94410.0  5.620  6.655188e+06  34668.563588  POINT (34668.564 6655187.820)

[189 rows x 5 columns]
gt1l2
wind

     AT_dist     SD          lats          lons                       geometry
115  13600.0  1.055  6.734870e+06  46588.379499  POINT (46588.379 6734869.558)
116  13700.0  1.060  6.734773e+06  46566.027724  POINT (46566.028 6734772.674)
118  13900.0  1.260  6.734489e+06  46499.247701  POINT (46499.248 6734489.492)
375  39600.0  0.975  6.709483e+06  40688.887348  POINT (40688.887 6709482.663)
376  39700.0  1.050  6.709366e+06  40661.321710  POINT (40661.322 6709366.415)
..       ...    ...           ...           ...                            ...
858  87900.0  3.320  6.662304e+06  29699.437959  POINT (29699.438 6662303.785)
859  88000.0  2.790  6.662304e+06  29699.437959  POINT (29699.438 6662303.785)
872  89300.0  5.200  6.661201e+06  29442.242234  POINT (29442.242 6661200.580)
873  89400.0  4.910  6.660734e+06  29332.285575  POINT (29332.286 6660733.734)
874  89500.0  5.055  6.660734e+06  29332.285575  POINT (29332.286 6660733.734)

[113 rows x 5 columns]
gt3l2
windows 573
     AT_di

In [ ]:
df